In [1]:
from gurobipy import*
import pandas as pd
import numpy as np
import math
import csv

#y = ty * 0.00000899823754
#x = 121 + (tx - 250000) * 0.000008983152841195214 / math.cos(math.radians(y))

# Setting

In [2]:
r= 500
#r=500m
n=156 #取幾個景點

data_A = pd.read_csv('site_twd97.csv');
data_S = pd.read_csv('PSO_twd97.csv');


weight = list(data_A['weight']);

#for i in range(0,len(data_A['經緯'])):
#    data_A['經緯'][i] = eval(data_A['經緯'][i])


#A景點
#S候選點

A=[]
for i in range(0,len(data_A)):
    A.append((data_A['TWD97 TM2 經'][i],data_A['TWD97 TM2 緯'][i]))

#S = [(22.98497, 120.19106),(22.987000, 120.204510),(22.981255, 120.205152),(22.98387, 120.20839),(22.98687, 120.20762),(22.99092, 120.20267),(22.99086, 120.2047),(22.98816, 120.20959),(22.99682, 120.20105),(22.99836, 120.20262),(23.00223, 120.16108),(23.00112, 120.16053),(22.99472, 120.20513),(22.98939, 120.19775),(22.98796, 120.20642),(22.98425, 120.20651)]


S = []
#for i in range(0,len(data_S)):    
#    tx=data_S['x'][i]
#    ty=data_S['y'][i]
#    y = ty * 0.00000899823754
#    x = 121 + (tx - 250000) * 0.000008983152841195214 / math.cos(math.radians(y))
for i in range(0,len(data_S)):
    S.append((data_S['x'][i],data_S['y'][i]))



In [3]:
len(S)

100

# Model

In [4]:
m = Model("Ucar")

M = 9999999999999999999
e = 0.00000000001
v = {};
alpha_p ={};
alpha_m ={};
beta_p ={};
beta_m ={};
vab={};
for i in range(1, n+1):
    for j in range(1,len(S)+1):
        v[i,j] = m.addVar(vtype = 'B',name='v[%s,%s]'%(i,j));
        alpha_p[i,j] = m.addVar(lb=0,vtype = 'S',name='alpha_p[%s,%s]'%(i,j));
        alpha_m[i,j] = m.addVar(lb=0,vtype = 'S',name='alpha_m[%s,%s]'%(i,j));
        beta_p[i,j] = m.addVar(lb=0,vtype = 'S',name='beta_p[%s,%s]'%(i,j));
        beta_m[i,j] = m.addVar(lb=0,vtype = 'S',name='beta_m[%s,%s]'%(i,j));
        vab[i,j] = m.addVar(lb=0,vtype = 'S',name='vab[%s,%s]'%(i,j));
    
        
m.update()


Academic license - for non-commercial use only


In [5]:
idx = 0
for i in range(1, n+1):
    for j in range(1,len(S)+1):
        idx += 1;
        m.addConstr( alpha_p[i,j]+alpha_m[i,j]+beta_p[i,j]+beta_m[i,j] - M*(1-v[i,j]) <= vab[i,j],'C1.1.%s'%idx);
        m.addConstr( vab[i,j] <= alpha_p[i,j]+alpha_m[i,j]+beta_p[i,j]+beta_m[i,j] + M*(1-v[i,j]),'C1.2.%s'%idx);
        m.addConstr( (-M)*v[i,j] <= vab[i,j],'C2.1.%s'%idx);
        m.addConstr( vab[i,j] <= M*v[i,j],'C2.2.%s'%idx);

        m.addConstr( alpha_p[i,j]-alpha_m[i,j] == abs(A[i-1][0]-S[j-1][0]),'C3.%s'%idx);
        m.addConstr( beta_p[i,j]-beta_m[i,j] == abs(A[i-1][1]-S[j-1][1]),'C4.%s'%idx);
        
        m.addConstr( (-M)*v[i,j]+e<=alpha_p[i,j]+alpha_m[i,j]+beta_p[i,j]+beta_m[i,j]-r,'C5.1.%s'%idx);
        m.addConstr( alpha_p[i,j]+alpha_m[i,j]+beta_p[i,j]+beta_m[i,j]-r<=M*(1-v[i,j])+e,'C5.2.%s'%idx);
        
for i in range(1,n+1):
    m.addConstr( quicksum(v[i,j] for j in range(1,len(S)+1) )>=1, 'C6.%s'%idx);

#m.addConstr( quicksum(v[i,j] for i in range(1,n+1) for j in range(1,len(S)+1) ) >=30, 'C6.%s'%idx);
        
        
        
m.setObjective(quicksum(weight[i-1]*quicksum(vab[i,j]for j in range(1,len(S)+1))for i in range(1,n+1)),GRB.MINIMIZE);




In [6]:
m.optimize();

Optimize a model with 124956 rows, 93600 columns and 483600 nonzeros
Variable types: 0 continuous, 15600 integer (15600 binary)
Semi-Variable types: 78000 continuous, 0 integer
Coefficient statistics:
  Matrix range     [1e+00, 1e+19]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-02, 1e+19]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.08 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


# Result

In [7]:
if m.status == GRB.Status.INFEASIBLE:
    print("INFEASIBLE")
    
else:

    print("Runtime:",m.Runtime,"\n")

    loc_ctr=[] #count for how many locations have been selected
    
    p=0
    for i in range(1,n+1):
        ctr=0
        for j in range(1,len(S)+1):
            if v[i,j].x > 0:
                ctr=ctr+1
                p=j
                loc_ctr.append(p)
                
                x1 = A[i-1][0]
                y1 = A[i-1][1]
                x2 = S[p-1][0]
                y2 = S[p-1][1]
                
                math.sqrt((x1-x2)**2+(y1-y2)**2)
                print("Site", i-1 , ": " , ctr, "   location(s)  at spot",p-1 , "  distance: ", math.sqrt((x1-x2)**2+(y1-y2)**2) , " m ")

    print("\n","all locations: ",len(S));      
    print("locations have been selected:",len(set(loc_ctr)),"\n");   

        

Runtime: 0.08981490135192871 

Site 0 :  1    location(s)  at spot 16   distance:  807.1020833820651  m 
Site 1 :  1    location(s)  at spot 88   distance:  1326.6136327506  m 
Site 2 :  1    location(s)  at spot 17   distance:  1117.0275692658356  m 
Site 3 :  1    location(s)  at spot 43   distance:  817.4492549999356  m 
Site 4 :  1    location(s)  at spot 99   distance:  860.6796312216484  m 
Site 5 :  1    location(s)  at spot 83   distance:  113.55776151381215  m 
Site 6 :  1    location(s)  at spot 15   distance:  866.1078492314102  m 
Site 7 :  1    location(s)  at spot 18   distance:  49.17278617282354  m 
Site 8 :  1    location(s)  at spot 82   distance:  1030.561177465689  m 
Site 9 :  1    location(s)  at spot 99   distance:  892.3868392685936  m 
Site 10 :  1    location(s)  at spot 91   distance:  1005.3266233915805  m 
Site 11 :  1    location(s)  at spot 19   distance:  728.6639022897077  m 
Site 12 :  1    location(s)  at spot 46   distance:  0.21189620095977957  m 
S

Site 120 :  1    location(s)  at spot 89   distance:  855.723630210029  m 
Site 121 :  1    location(s)  at spot 19   distance:  1073.774884880438  m 
Site 122 :  1    location(s)  at spot 87   distance:  1050.2337100856741  m 
Site 123 :  1    location(s)  at spot 84   distance:  939.3025009016226  m 
Site 124 :  1    location(s)  at spot 24   distance:  910.055089376239  m 
Site 125 :  1    location(s)  at spot 19   distance:  766.3428718922894  m 
Site 126 :  1    location(s)  at spot 22   distance:  922.4492182228421  m 
Site 127 :  1    location(s)  at spot 29   distance:  1.4751271131327692  m 
Site 128 :  1    location(s)  at spot 93   distance:  926.1084238899656  m 
Site 129 :  1    location(s)  at spot 71   distance:  1.5592626465139978  m 
Site 130 :  1    location(s)  at spot 93   distance:  534.9978490610185  m 
Site 131 :  1    location(s)  at spot 94   distance:  697.8824434675647  m 
Site 132 :  1    location(s)  at spot 23   distance:  8.53696081747883  m 
Site 133 :  

# find err


In [8]:
if m.status == GRB.Status.INFEASIBLE:
        print('Optimization was stopped with status %d' % m.status)
        # do IIS, find infeasible constraints
        m.computeIIS()
        for c in m.getConstrs():
            if c.IISConstr:
                print('%s' % c.constrName)
